# Importing Brevitas networks into FINN with the QONNX interchange format

**Note: This notebook is very similar to the 1a notebook, in that it shows the same concepts for the QONNX ingestion as 1a does for FINN-ONNX.**

In this notebook we'll go through an example of how to import a Brevitas-trained QNN into FINN. The steps will be as follows:

1. Load up the trained PyTorch model
2. Call Brevitas QONNX export and visualize with Netron
3. Import into FINN and converting QONNX to FINN-ONNX

We'll use the following utility functions to print the source code for function calls (`showSrc()`) and to visualize a network using netron (`showInNetron()`) in the Jupyter notebook:

In [ ]:
import onnx
from finn.util.visualization import showSrc, showInNetron

## 1. Load up the trained PyTorch model

The FINN Docker image comes with several [example Brevitas networks](https://github.com/Xilinx/brevitas/tree/master/src/brevitas_examples/bnn_pynq), and we'll use the LFC-w1a1 model as the example network here. This is a binarized fully connected network trained on the MNIST dataset. Let's start by looking at what the PyTorch network definition looks like:

In [ ]:
from brevitas_examples import bnn_pynq
showSrc(bnn_pynq.models.FC)

We can see that the network topology is constructed using a few helper functions that generate the quantized linear layers and quantized activations. The bitwidth of the layers is actually parametrized in the constructor, so let's instantiate a 1-bit weights and activations version of this network. We also have pretrained weights for this network, which we will load into the model.

In [ ]:
from finn.util.test import get_test_model
lfc = get_test_model(netname = "LFC", wbits = 1, abits = 1, pretrained = True)
lfc

We have now instantiated our trained PyTorch network. Let's try to run an example MNIST image through the network using PyTorch.

In [ ]:
import torch
import matplotlib.pyplot as plt
from pkgutil import get_data
import onnx
import onnx.numpy_helper as nph
raw_i = get_data("qonnx.data", "onnx/mnist-conv/test_data_set_0/input_0.pb")
input_tensor = onnx.load_tensor_from_string(raw_i)
input_tensor_npy = nph.to_array(input_tensor)
input_tensor_pyt = torch.from_numpy(input_tensor_npy).float()
imgplot = plt.imshow(input_tensor_npy.reshape(28,28), cmap='gray')

In [ ]:
from torch.nn.functional import softmax
# do forward pass in PyTorch/Brevitas
produced = lfc.forward(input_tensor_pyt).detach()
probabilities = softmax(produced, dim=-1).flatten()
probabilities

In [ ]:
import numpy as np
objects = [str(x) for x in range(10)]
y_pos = np.arange(len(objects))
plt.bar(y_pos, probabilities, align='center', alpha=0.5)
plt.xticks(y_pos, objects)
plt.ylabel('Predicted Probability')
plt.title('LFC-w1a1 Predictions for Image')
plt.show()

## 2. Call Brevitas QONNX export and visualize with Netron

Brevitas comes with built-in QONNX export functionality. This is similar to the regular ONNX export capabilities of PyTorch, with a few differences:

1. Weight and activation quantization is represented as a 'fake-quantization' with Quant and BipolarQuant nodes.
2. Truncation operations as required by average pooling are represented with a Trunc node.

One can read more about how QONNX works and why it was developed here: https://xilinx.github.io/finn//2021/11/03/qonnx-and-finn.html

Additionally QONNX comes with a set of tools for working with the format. These are maintained together with the Fast Machinelearning collaboration as an open-source projet here: https://github.com/fastmachinelearning/qonnx

It's actually quite straightforward to export QONNX from our Brevitas model as follows:

In [ ]:
from brevitas.export.onnx.generic.manager import BrevitasONNXManager
export_onnx_path = "/tmp/LFCW1A1_qonnx.onnx"
input_shape = (1, 1, 28, 28)
BrevitasONNXManager.export(lfc, input_shape, export_onnx_path);

Let's examine what the exported ONNX model looks like. For this, we will use the Netron visualizer:

In [ ]:
showInNetron(export_onnx_path)

When running this notebook in the FINN Docker container, you should be able to see an interactive visualization of the imported network above, and click on individual nodes to inspect their parameters. If you look at any of the MatMul nodes, you should be able to see that the weights are all {-1, +1} values.

## 3. Import into FINN and converting QONNX to FINN-ONNX

Similarily to the 1a notebook we will first run a cleanup transformation on the exported QONNX model.

In [ ]:
from qonnx.util.cleanup import cleanup

export_onnx_path_cleaned = "/tmp/LFCW1A1-qonnx-clean.onnx"
cleanup(export_onnx_path, out_file=export_onnx_path_cleaned)

In [ ]:
showInNetron(export_onnx_path_cleaned)

We will now import this QONNX model into FINN using the ModelWrapper. Here we can immediatley execute the model to verify correctness.

In [ ]:
from qonnx.core.modelwrapper import ModelWrapper
import qonnx.core.onnx_exec as oxe
model = ModelWrapper(export_onnx_path_cleaned)
input_dict = {"0": nph.to_array(input_tensor)}
output_dict = oxe.execute_onnx(model, input_dict)
produced_qonnx = output_dict[list(output_dict.keys())[0]]

produced_qonnx

In [ ]:
np.isclose(produced, produced_finn).all()

Using the `QONNXtoFINN` transformation we can convert the model to the FINN internal FINN-ONNX representation. Notably all Quant and BipolarQuant nodes will have disappeared and are converted into MultiThreshold nodes.

In [ ]:
from finn.transformation.qonnx.convert_qonnx_to_finn import ConvertQONNXtoFINN
model = ModelWrapper(export_onnx_path_cleaned)

model = model.transform(ConvertQONNXtoFINN())

export_onnx_path_converted = "/tmp/LFCW1A1-qonnx-converted.onnx"
model.save(export_onnx_path_converted)

In [ ]:
showInNetron(export_onnx_path_converted)

And once again we can execute the model with the FINN/QONNX execution engine.

In [ ]:
model = ModelWrapper(export_onnx_path_cleaned)
input_dict = {"0": nph.to_array(input_tensor)}
output_dict = oxe.execute_onnx(model, input_dict)
produced_finn = output_dict[list(output_dict.keys())[0]]

produced_finn

In [ ]:
np.isclose(produced, produced_finn).all()

We have succesfully verified that the transformed and cleaned-up FINN graph still produces the same output, and can now use this model for further processing in FINN.